In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import df_from_csv, to_be_checked, is_corr_ok, plot_hist, plot_boxes, plot_lines, df_to_csv, drop_pm25_outliers

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## PM2.5 Clean Data

In [2]:
# Load Raw Data
df_pm25_raw = df_from_csv("pm25_raw.csv")

# Check Data Types
print(df_pm25_raw.dtypes)

df_pm25_raw.head()

TANGARA_260A    float64
TANGARA_4B1A    float64
TANGARA_14D6    float64
TANGARA_2B42    float64
TANGARA_2E9A    float64
TANGARA_2FF6    float64
TANGARA_307A    float64
TANGARA_48C6    float64
TANGARA_F1AE    float64
TANGARA_06BE    float64
dtype: object


,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,
2023-11-10 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN
2023-11-10 00:00:30-05:00,8.0,6.0,6.0,NaN,1.0,9.0,9.0,1.0,6.0,9.0
2023-11-10 00:01:00-05:00,7.0,4.0,6.0,NaN,1.0,8.0,9.0,1.0,5.0,9.0
2023-11-10 00:01:30-05:00,8.0,5.0,7.0,NaN,0.0,8.0,9.0,2.0,5.0,9.0
2023-11-10 00:02:00-05:00,7.0,3.0,6.0,NaN,0.0,8.0,8.0,2.0,5.0,9.0


## Descriptive Statistics

In [3]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5615.000000,5663.000000,1039.000000,4226.000000,5679.000000,5692.00000,5599.000000,5511.000000,5734.000000,5388.000000
mean,8.179163,9.315027,9.115496,6.977047,6.701004,7.50000,8.666548,6.022319,7.931287,9.582962
std,4.535844,4.566012,3.431429,5.910071,6.001325,4.51902,5.084152,3.974745,4.721420,5.822508
min,1.000000,1.000000,3.000000,0.000000,0.000000,1.00000,1.000000,0.000000,1.000000,1.000000
25%,4.000000,7.000000,6.000000,2.000000,2.000000,5.00000,7.000000,3.000000,5.000000,6.000000
50%,8.000000,9.000000,9.000000,7.000000,5.000000,7.00000,8.000000,6.000000,7.000000,9.000000
75%,11.000000,11.000000,12.000000,10.000000,10.000000,9.25000,10.000000,8.000000,9.000000,12.000000
max,27.000000,68.000000,23.000000,32.000000,48.000000,46.00000,173.000000,32.000000,73.000000,94.000000


## Missing Data

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_pm25_raw.columns:
    df_pm25_sensor = df_pm25_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_pm25_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_14D6, Data: 18%, Missing: 82%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 73%, Missing: 27%, To be checked


## Data Correlation

In [5]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_pm25_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_pm25_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_pm25_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_pm25_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")

Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


## Histograms

In [6]:
# Plot Histograms
plot_hist(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Boxplots

In [7]:
# Plot Boxplots
plot_boxes(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Lineplots

In [8]:
# Plot Lineplots
plot_lines(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Drop PM2.5 Outliers

In [9]:
# Drop PM2.5 Outliers
df_pm25_clean, resume = drop_pm25_outliers(df_pm25_raw)

In [10]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5541.000000,5427.000000,1038.000000,4079.000000,5467.000000,5340.000000,5211.000000,5323.000000,5355.000000,5127.000000
mean,7.969681,8.674774,9.102119,6.257906,5.950430,6.739326,7.794089,5.600413,7.017180,8.779598
std,4.182716,2.988458,3.405874,4.583022,4.692591,3.428480,2.668220,3.309623,2.961981,4.429641
min,1.000000,1.000000,3.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,4.000000,7.000000,6.000000,1.000000,2.000000,5.000000,6.000000,3.000000,5.000000,6.000000
50%,8.000000,9.000000,9.000000,7.000000,5.000000,7.000000,8.000000,5.000000,7.000000,9.000000
75%,10.000000,10.000000,12.000000,9.000000,9.000000,9.000000,9.000000,8.000000,9.000000,11.000000
max,21.000000,17.000000,20.000000,22.000000,22.000000,15.000000,14.000000,15.000000,15.000000,21.000000


In [11]:
# Describe Data
df_pm25_clean.describe()

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5541.000000,5427.000000,1038.000000,4079.000000,5467.000000,5340.000000,5211.000000,5323.000000,5355.000000,5127.000000
mean,7.969681,8.674774,9.102119,6.257906,5.950430,6.739326,7.794089,5.600413,7.017180,8.779598
std,4.182716,2.988458,3.405874,4.583022,4.692591,3.428480,2.668220,3.309623,2.961981,4.429641
min,1.000000,1.000000,3.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,4.000000,7.000000,6.000000,1.000000,2.000000,5.000000,6.000000,3.000000,5.000000,6.000000
50%,8.000000,9.000000,9.000000,7.000000,5.000000,7.000000,8.000000,5.000000,7.000000,9.000000
75%,10.000000,10.000000,12.000000,9.000000,9.000000,9.000000,9.000000,8.000000,9.000000,11.000000
max,21.000000,17.000000,20.000000,22.000000,22.000000,15.000000,14.000000,15.000000,15.000000,21.000000


## Histograms

In [12]:
# Plot Histograms
plot_hist(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Boxplots

In [13]:
# Plot Boxplots
plot_boxes(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Lineplots

In [14]:
# Plot Lineplots
plot_lines(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


In [15]:
# Save PM2.5 Data Frame Sensors into CSV file
df_to_csv(df_pm25_clean, "pm25_clean.csv", datafolder='1_clean')